In [1]:
from pathlib import Path

import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

In [2]:
model_id = "/home/jovyan/nfs_share/models/InternVL3-8B"
# here, specify the name of the image
imageName = "/home/jovyan/nfs_share/tod/datasets/synthetic_invoice_014.png"

model = AutoModel.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True
)

# open the image
image = Image.open(imageName)

FlashAttention2 is not installed.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
# InternVL3 uses a simpler prompt format
question = "How much did Jessica pay?"

In [ ]:
# Process image for InternVL3
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

# InternVL3 image processing
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
image_size = 448

transform = T.Compose([
    T.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img),
    T.Resize((image_size, image_size), interpolation=InterpolationMode.BICUBIC),
    T.ToTensor(),
    T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

pixel_values = transform(image).unsqueeze(0)  # Shape: [1, 3, 448, 448]
if torch.cuda.is_available():
    pixel_values = pixel_values.cuda().to(torch.float16)
else:
    pixel_values = pixel_values.to(torch.float32)

# Generate response using InternVL3
response = model.chat(
    tokenizer=tokenizer,
    pixel_values=pixel_values,
    question=question,
    generation_config={
        "max_new_tokens": 2000,
        "do_sample": True,
        "temperature": 0.7
    }
)

print(response)

In [ ]:
# save the answer in a file
with Path("/home/jovyan/nfs_share/tod/output/internvl3_output.txt").open(
    "w", encoding="utf-8"
) as text_file:
    text_file.write(response)